## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-17-17-11-19 +0000,nyt,A Deadly Medieval Path in England Claims a Mod...,https://www.nytimes.com/2026/02/17/world/europ...
1,2026-02-17-17-10-47 +0000,nyt,"Ed Crane, Who Built a Libertarian Stronghold, ...",https://www.nytimes.com/2026/02/17/us/politics...
2,2026-02-17-17-09-31 +0000,cbc,"Cucumber costs are skyrocketing in Russia, and...",https://www.cbc.ca/news/world/russia-cucumber-...
3,2026-02-17-17-08-37 +0000,nyt,The Opera ‘Monster’s Paradise’ Takes Satirical...,https://www.nytimes.com/2026/02/17/arts/music/...
4,2026-02-17-17-07-56 +0000,nyt,"Al Sharpton, Bernice King and Trump Among Trib...",https://www.nytimes.com/2026/02/17/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2480/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
29,trump,29
37,jackson,16
36,jesse,14
98,new,14
101,talks,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
163,2026-02-17-09-48-00 +0000,wsj,"Jesse Jackson, the civil-rights activist and p...",https://www.wsj.com/politics/jesse-jackson-civ...,84
99,2026-02-17-13-26-00 +0000,nypost,"Iran’s supreme leader taunts Trump, US as high...",https://nypost.com/2026/02/17/us-news/irans-su...,79
106,2026-02-17-12-51-03 +0000,nypost,Warner Bros. Discovery will restart talks with...,https://nypost.com/2026/02/17/business/warner-...,69
4,2026-02-17-17-07-56 +0000,nyt,"Al Sharpton, Bernice King and Trump Among Trib...",https://www.nytimes.com/2026/02/17/us/politics...,68
33,2026-02-17-16-30-16 +0000,nypost,RI trans shooter Robert Dorgan’s hockey-player...,https://nypost.com/2026/02/17/us-news/ri-trans...,63


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
163,84,2026-02-17-09-48-00 +0000,wsj,"Jesse Jackson, the civil-rights activist and p...",https://www.wsj.com/politics/jesse-jackson-civ...
99,76,2026-02-17-13-26-00 +0000,nypost,"Iran’s supreme leader taunts Trump, US as high...",https://nypost.com/2026/02/17/us-news/irans-su...
106,56,2026-02-17-12-51-03 +0000,nypost,Warner Bros. Discovery will restart talks with...,https://nypost.com/2026/02/17/business/warner-...
33,55,2026-02-17-16-30-16 +0000,nypost,RI trans shooter Robert Dorgan’s hockey-player...,https://nypost.com/2026/02/17/us-news/ri-trans...
12,38,2026-02-17-16-51-48 +0000,nyt,Ukraine and Russia Hold New Round of Peace Tal...,https://www.nytimes.com/2026/02/17/world/europ...
88,33,2026-02-17-14-00-30 +0000,nypost,Pulse-pounding video shows heroes wrestle tran...,https://nypost.com/2026/02/17/us-news/video-sh...
116,31,2026-02-17-12-23-37 +0000,nypost,Top-performing fund warns software firms face ...,https://nypost.com/2026/02/17/business/top-per...
112,30,2026-02-17-12-41-09 +0000,bbc,Nancy Guthrie's family cleared as possible sus...,https://www.bbc.com/news/articles/cjd93rl849xo...
177,30,2026-02-17-06-59-28 +0000,nyt,"Tom Pritzker, Hyatt Heir, Steps Down as Execut...",https://www.nytimes.com/2026/02/16/us/politics...
60,27,2026-02-17-15-22-49 +0000,nypost,Head of multibillion-dollar California High-Sp...,https://nypost.com/2026/02/17/us-news/head-of-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
